### Decimal to Binary Conversion

Single layer feedforward neural network that implements decimal to binary conversion.

In [498]:
%run core.ipynb

In [499]:
import numpy as np


def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def dx_sigmoid(x):
    return sigmoid(x) * (1.0 - sigmoid(x))


def int2dec(n):
    return np.array(tuple(0.99 if i == n else 0.01 for i in range(10)))


def dec2int(X):
    for i, x in enumerate(X):
        if x > 0.5:
            return i

def int2bin(n):
    b = []
    while n > 0:
        n, x = divmod(n, 2)
        b.append(x)
    return np.array(b + [0]*(4 - len(b)))


def bin2int(X):
    return sum(2 ** i if x > 0.5 else 0 for i, x in enumerate(X))


In [500]:
for _ in range(3):
    n = random.randint(0, 10)
    print(f"digit:		{n}")
    print(f"decimal:	{int2dec(n)}")
    print(f"binary:		{int2bin(n)}")
    assert dec2int(int2dec(n)) == n
    assert bin2int(int2bin(n)) == n

digit:		3
decimal:	[0.01 0.01 0.01 0.99 0.01 0.01 0.01 0.01 0.01 0.01]
binary:		[1 1 0 0]
digit:		9
decimal:	[0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.99]
binary:		[1 0 0 1]
digit:		6
decimal:	[0.01 0.01 0.01 0.01 0.01 0.01 0.99 0.01 0.01 0.01]
binary:		[0 1 1 0]


#### Manual Weights and Biasis

In [501]:
def get_dec2bin_weights_and_biases():
    A = []
    for i in range(10):
        A.append(int2bin(i))
    W = np.transpose(np.array(A))
    B = - (sigmoid(0.99) - sigmoid(0.01)) / 2 * np.ones(4)
    return W, B


In [502]:
def check_dec2bin(W, B):
    for i in range(10):
        X = int2dec(i)
        Y = sigmoid(np.dot(W, X) + B)
        x, y = dec2int(X), bin2int(Y)
        Y = np.reshape(Y, (4,))
        assert x == y
        print(f'{x} -> {Y} = {[1 if t > 0.5 else 0 for t in Y]}')


In [503]:
W, B = get_dec2bin_weights_and_biases()
check_dec2bin(W, B)

0 -> [0.48418179 0.48168471 0.48168471 0.47669341] = [0, 0, 0, 0]
1 -> [0.71437046 0.48168471 0.48168471 0.47669341] = [1, 0, 0, 0]
2 -> [0.48418179 0.71232564 0.48168471 0.47669341] = [0, 1, 0, 0]
3 -> [0.71437046 0.71232564 0.48168471 0.47669341] = [1, 1, 0, 0]
4 -> [0.48418179 0.48168471 0.71232564 0.47669341] = [0, 0, 1, 0]
5 -> [0.71437046 0.48168471 0.71232564 0.47669341] = [1, 0, 1, 0]
6 -> [0.48418179 0.71232564 0.71232564 0.47669341] = [0, 1, 1, 0]
7 -> [0.71437046 0.71232564 0.71232564 0.47669341] = [1, 1, 1, 0]
8 -> [0.48418179 0.48168471 0.48168471 0.70820995] = [0, 0, 0, 1]
9 -> [0.71437046 0.48168471 0.48168471 0.70820995] = [1, 0, 0, 1]


#### Weights and Biasis using Delta Rule

In [504]:
from delta_rule import delta_rule


def get_dec2bin_delta_rule_test_data():
    D = {n: int2dec(n) for n in range(10)}
    B = {n: int2bin(n) for n in range(10)}
    N = []
    for i in range(4):
        N.append([(D[n], B[n][i]) for n in range(10)])
    return N


def get_dec2bin_delta_rule_weights_and_biases(a, da, s=0.01, epochs=50):
    W, B = [], []
    for T in get_dec2bin_delta_rule_test_data():
        w, b = delta_rule(T, a=a, da=da, s=s, epochs=epochs)
        W.append(w)
        B.append(b)
    return np.array(W), np.array(B)


def check_dec2bin_delta_rule(W, B, a, classify):
    for n in range(10):
        X = tuple(int2bin(n))
        Y = tuple(a(x) for x in np.dot(W, int2dec(n)) + B)
        Z = tuple(classify(t) for t in Y)
        if X == Z:
            print(f"{n} =	{_rnd(Y, n=2)} =	{Z}")
        else:
            print(_red(f"{n} ≠	{_rnd(Y, n=2)} =	{Z}	≠	{X}"))


In [505]:
get_dec2bin_delta_rule_test_data()[0]

[(array([0.99, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]), 0),
 (array([0.01, 0.99, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]), 1),
 (array([0.01, 0.01, 0.99, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]), 0),
 (array([0.01, 0.01, 0.01, 0.99, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]), 1),
 (array([0.01, 0.01, 0.01, 0.01, 0.99, 0.01, 0.01, 0.01, 0.01, 0.01]), 0),
 (array([0.01, 0.01, 0.01, 0.01, 0.01, 0.99, 0.01, 0.01, 0.01, 0.01]), 1),
 (array([0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.99, 0.01, 0.01, 0.01]), 0),
 (array([0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.99, 0.01, 0.01]), 1),
 (array([0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.99, 0.01]), 0),
 (array([0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.99]), 1)]

In [506]:
a, da, classify = linear, dx_linear, lambda x: heaviside(x, offset=0.5)
s, epochs = 0.01, 50

W, B = get_dec2bin_delta_rule_weights_and_biases(a, da, s=s, epochs=epochs)
check_dec2bin_delta_rule(W, B, a, classify)

0 =	(0.31, 0.25, 0.25, 0.13) =	(0, 0, 0, 0)
1 =	(0.7, 0.25, 0.25, 0.13) =	(1, 0, 0, 0)
2 =	(0.31, 0.63, 0.25, 0.13) =	(0, 1, 0, 0)
3 =	(0.7, 0.63, 0.25, 0.13) =	(1, 1, 0, 0)
4 =	(0.31, 0.25, 0.64, 0.13) =	(0, 0, 1, 0)
5 =	(0.7, 0.25, 0.64, 0.13) =	(1, 0, 1, 0)
6 =	(0.31, 0.63, 0.63, 0.13) =	(0, 1, 1, 0)
7 =	(0.7, 0.63, 0.63, 0.14) =	(1, 1, 1, 0)
8 =	(0.31, 0.25, 0.25, 0.52) =	(0, 0, 0, 1)
9 =	(0.7, 0.25, 0.25, 0.52) =	(1, 0, 0, 1)


In [507]:
a, da, classify = relu, dx_relu, lambda x: heaviside(x, offset=0.5)
s, epochs = 0.01, 50

W, B = get_dec2bin_delta_rule_weights_and_biases(a, da, s=s, epochs=epochs)
check_dec2bin_delta_rule(W, B, a, classify)

0 =	(0.31, 0.25, 0.25, 0.13) =	(0, 0, 0, 0)
1 =	(0.7, 0.25, 0.25, 0.13) =	(1, 0, 0, 0)
2 =	(0.31, 0.63, 0.25, 0.13) =	(0, 1, 0, 0)
3 =	(0.7, 0.63, 0.25, 0.13) =	(1, 1, 0, 0)
4 =	(0.31, 0.25, 0.64, 0.13) =	(0, 0, 1, 0)
5 =	(0.7, 0.25, 0.64, 0.13) =	(1, 0, 1, 0)
6 =	(0.31, 0.63, 0.64, 0.13) =	(0, 1, 1, 0)
7 =	(0.7, 0.63, 0.63, 0.13) =	(1, 1, 1, 0)
8 =	(0.31, 0.25, 0.25, 0.52) =	(0, 0, 0, 1)
9 =	(0.7, 0.25, 0.25, 0.51) =	(1, 0, 0, 1)


In [508]:
a, da, classify = sigmoid, dx_sigmoid, lambda x: heaviside(x, offset=0.5)
s, epochs = 0.01, 750

W, B = get_dec2bin_delta_rule_weights_and_biases(a, da, s=s, epochs=epochs)
check_dec2bin_delta_rule(W, B, a, classify)

0 =	(0.33, 0.3, 0.29, 0.2) =	(0, 0, 0, 0)
1 =	(0.67, 0.3, 0.29, 0.2) =	(1, 0, 0, 0)
2 =	(0.33, 0.61, 0.29, 0.2) =	(0, 1, 0, 0)
3 =	(0.67, 0.61, 0.29, 0.2) =	(1, 1, 0, 0)
4 =	(0.33, 0.3, 0.62, 0.2) =	(0, 0, 1, 0)
5 =	(0.67, 0.3, 0.62, 0.2) =	(1, 0, 1, 0)
6 =	(0.33, 0.61, 0.62, 0.2) =	(0, 1, 1, 0)
7 =	(0.67, 0.61, 0.62, 0.2) =	(1, 1, 1, 0)
8 =	(0.33, 0.3, 0.29, 0.5) =	(0, 0, 0, 1)
9 =	(0.67, 0.3, 0.29, 0.5) =	(1, 0, 0, 1)


#### Weights and Biasis using Perceptron

In [509]:
get_dec2bin_perceptron_test_data = get_dec2bin_delta_rule_test_data


def get_dec2bin_perceptron_weights_and_biases(s=0.01, epochs=50):
    W, B = [], []
    for T in get_dec2bin_perceptron_test_data():
        w, b = perceptron(T, s=s, epochs=epochs)
        W.append(w)
        B.append(b)
    return np.array(W), np.array(B)


def check_dec2bin_perceptron(W, B):
    for n in range(10):
        X = tuple(int2bin(n))
        Y = tuple(np.dot(W, int2dec(n)) + B)
        Z = tuple(heaviside(t) for t in Y)
        if X == Z:
            print(f"{n} =	{_rnd(Y, n=3)} =	{Z}")
        else:
            print(_red(f"{n} ≠	{_rnd(Y, n=3)} =	{Z}	≠	{X}"))


In [510]:
s, epochs = 0.001, 6

W, B = get_dec2bin_perceptron_weights_and_biases(s=s, epochs=epochs)
check_dec2bin_perceptron(W, B)

0 =	(-0.001, -0.001, -0.001, -0.002) =	(0, 0, 0, 0)
1 =	(0.001, -0.001, -0.001, -0.002) =	(1, 0, 0, 0)
2 =	(-0.001, 0.002, -0.001, -0.002) =	(0, 1, 0, 0)
3 =	(0.001, 0.002, -0.001, -0.002) =	(1, 1, 0, 0)
4 =	(-0.001, -0.001, 0.002, -0.002) =	(0, 0, 1, 0)
5 =	(0.001, -0.001, 0.002, -0.001) =	(1, 0, 1, 0)
6 =	(-0.001, 0.002, 0.002, -0.001) =	(0, 1, 1, 0)
7 =	(0.001, 0.0, 0.0, -0.001) =	(1, 1, 1, 0)
8 =	(-0.001, -0.001, -0.001, 0.001) =	(0, 0, 0, 1)
9 =	(0.001, -0.001, -0.001, 0.001) =	(1, 0, 0, 1)
